# Azure AI Search आणि NVIDIA NIM आणि LlamaIndex एकत्रीकरण

या नोटबुकमध्ये, NVIDIA च्या AI मॉडेल्स आणि LlamaIndex चा उपयोग करून एक शक्तिशाली Retrieval-Augmented Generation (RAG) पाइपलाइन कशी तयार करायची हे दाखवले जाईल. आपण NVIDIA च्या LLMs आणि embeddings वापरू, त्यांना Azure AI Search सोबत एकत्रित करू आणि RAG चा उपयोग करून शोधाची गुणवत्ता आणि कार्यक्षमता सुधारू.

## फायदे
- **स्केलेबिलिटी**: NVIDIA च्या मोठ्या भाषा मॉडेल्स आणि Azure AI Search चा उपयोग करून स्केलेबल आणि कार्यक्षम retrieval मिळवा.
- **खर्च कार्यक्षमता**: प्रभावी व्हेक्टर स्टोरेज आणि हायब्रिड शोध तंत्रज्ञानासह शोध आणि retrieval ऑप्टिमाइझ करा.
- **उच्च कार्यक्षमता**: शक्तिशाली LLMs आणि व्हेक्टराइज्ड शोध एकत्र करून जलद आणि अधिक अचूक प्रतिसाद मिळवा.
- **गुणवत्ता**: संबंधित retrieved दस्तऐवजांसह LLM प्रतिसादांना आधार देऊन उच्च शोध गुणवत्ता राखा.

## पूर्वअट
- 🐍 Python 3.9 किंवा त्याहून अधिक
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key NVIDIA च्या LLMs आणि Embeddings साठी NVIDIA NIM मायक्रोसर्व्हिसेसद्वारे प्रवेशासाठी

## समाविष्ट वैशिष्ट्ये
- ✅ NVIDIA LLM एकत्रीकरण (आपण [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) वापरणार आहोत)
- ✅ NVIDIA Embeddings (आपण [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) वापरणार आहोत)
- ✅ Azure AI Search प्रगत retrieval मोड्स
- ✅ LlamaIndex सह दस्तऐवज इंडेक्सिंग
- ✅ Azure AI Search आणि LlamaIndex सह NVIDIA LLMs वापरून RAG

चला सुरुवात करूया!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## स्थापना आणि आवश्यकता
Python आवृत्ती >3.10 वापरून Python वातावरण तयार करा.

## सुरुवात करा!


NVIDIA AI Foundation मॉडेल्स वापरण्यासाठी तुम्हाला `NVIDIA_API_KEY` आवश्यक आहे:
1) [NVIDIA](https://build.nvidia.com/explore/discover) वर एक मोफत खाते तयार करा.
2) तुमच्या पसंतीच्या मॉडेलवर क्लिक करा.
3) Input अंतर्गत, Python टॅब निवडा, आणि **Get API Key** वर क्लिक करा, त्यानंतर **Generate Key** वर क्लिक करा.
4) तयार केलेली key कॉपी करा आणि NVIDIA_API_KEY म्हणून सेव्ह करा. त्यानंतर, तुम्हाला endpoints वर प्रवेश मिळेल.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG उदाहरण LLM आणि एम्बेडिंग वापरून
### 1) LLM प्रारंभ करा
`llama-index-llms-nvidia`, ज्याला NVIDIA चा LLM कनेक्टर म्हणून ओळखले जाते, तुम्हाला NVIDIA API कॅटलॉगमध्ये उपलब्ध असलेल्या सुसंगत मॉडेल्सशी कनेक्ट होण्याची आणि त्यातून जनरेट करण्याची परवानगी देते. चॅट पूर्णता मॉडेल्सची यादी येथे पहा: https://build.nvidia.com/search?term=Text-to-Text

येथे आपण **mixtral-8x7b-instruct-v0.1** वापरणार आहोत


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) एम्बेडिंग प्रारंभ करा  
`llama-index-embeddings-nvidia`, ज्याला NVIDIA चा एम्बेडिंग्ज कनेक्टर म्हणूनही ओळखले जाते, तुम्हाला NVIDIA API कॅटलॉगमध्ये उपलब्ध असलेल्या सुसंगत मॉडेल्सशी कनेक्ट होण्याची आणि त्यातून निर्माण करण्याची परवानगी देते. आम्ही एम्बेडिंग मॉडेल म्हणून `nvidia/nv-embedqa-e5-v5` निवडले आहे. टेक्स्ट एम्बेडिंग मॉडेल्सची यादी येथे पहा: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) तुमच्या डेटावर प्रश्न विचारण्यासाठी एक क्वेरी इंजिन तयार करा

Azure AI Search मध्ये शुद्ध व्हेक्टर शोध वापरून आणि आमच्या LLM (Phi-3.5-MOE) वर उत्तर आधारित करण्यासाठी येथे एक क्वेरी आहे.


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Azure AI Search मध्ये हायब्रिड शोध वापरून एक प्रश्न येथे आहे.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### व्हेक्टर शोध विश्लेषण
LLM प्रतिसादाने रशियाच्या शेअर बाजारासंबंधित स्रोत मजकुरात नमूद केलेल्या महत्त्वाच्या आर्थिक परिणामांचे अचूकपणे वर्णन केले आहे. विशेषतः, यात नमूद केले आहे की रशियाच्या शेअर बाजारात मोठी घसरण झाली, ज्यामुळे त्याचे 40% मूल्य कमी झाले, आणि चालू परिस्थितीमुळे व्यापार थांबवण्यात आला. हा प्रतिसाद स्रोतामध्ये दिलेल्या माहितीसोबत चांगल्या प्रकारे जुळतो, ज्यामुळे हे स्पष्ट होते की LLM ने रशियाच्या कृती आणि लागू केलेल्या निर्बंधांमुळे शेअर बाजारावर झालेल्या परिणामांचे योग्य प्रकारे ओळख आणि संक्षेप केले आहे.

#### स्रोत नोड्स टिप्पणी
स्रोत नोड्समध्ये आंतरराष्ट्रीय निर्बंधांमुळे रशियाला भोगावे लागलेले आर्थिक परिणाम सविस्तरपणे मांडले आहेत. मजकुरात नमूद केले आहे की रशियाच्या शेअर बाजाराचे 40% मूल्य कमी झाले आणि व्यापार थांबवण्यात आला. याशिवाय, इतर आर्थिक परिणामांचा उल्लेख केला आहे, जसे की रुबलचे अवमूल्यन आणि रशियाच्या अर्थव्यवस्थेचे व्यापक पृथक्करण. LLM प्रतिसादाने या नोड्समधील महत्त्वाचे मुद्दे प्रभावीपणे संक्षेपित केले आहेत, विशेषतः शेअर बाजारावर झालेल्या परिणामांवर लक्ष केंद्रित केले आहे, जसे की चौकशीमध्ये विचारले होते.


आता, Hybrid Search योग्य आणि ठोस उत्तर देत नाही अशा एका प्रश्नावर नजर टाकूया:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### हायब्रिड शोध: LLM प्रतिसाद विश्लेषण
हायब्रिड शोधाच्या उदाहरणामधील LLM प्रतिसाद सूचित करतो की दिलेल्या संदर्भामध्ये रशियाने युक्रेनवर केलेल्या आक्रमणाची अचूक तारीख नमूद केलेली नाही. हा प्रतिसाद दर्शवतो की LLM स्रोत दस्तऐवजांमधील उपलब्ध माहितीचा उपयोग करत आहे, परंतु मजकुरामध्ये अचूक तपशील नसल्याचे मान्य करते.

प्रतिसाद योग्य आहे कारण तो ओळखतो की संदर्भामध्ये रशियाच्या आक्रमणाशी संबंधित घटना नमूद आहेत, परंतु विशिष्ट आक्रमणाची तारीख स्पष्टपणे दिलेली नाही. हे LLM च्या दिलेल्या माहितीचे आकलन करण्याच्या क्षमतेचे प्रदर्शन करते, तसेच मजकुरामधील उणीव ओळखण्याची क्षमता दाखवते. LLM प्रभावीपणे वापरकर्त्याला अचूक तारीख शोधण्यासाठी बाह्य स्रोत किंवा ऐतिहासिक नोंदींचा उपयोग करण्यास प्रवृत्त करते, जेव्हा माहिती अपूर्ण असते तेव्हा सावधगिरीचे स्तर दाखवते.

### स्रोत नोड्स विश्लेषण
हायब्रिड शोधाच्या उदाहरणामधील स्रोत नोड्समध्ये एका भाषणातील उतारे आहेत, ज्यामध्ये युक्रेनमधील रशियाच्या कृतींवर अमेरिकेच्या प्रतिसादावर चर्चा केली आहे. हे नोड्स व्यापक भू-राजकीय परिणाम आणि आक्रमणाच्या प्रतिसादात अमेरिका आणि तिच्या सहयोगींनी घेतलेल्या पावलांवर भर देतात, परंतु विशिष्ट आक्रमणाची तारीख नमूद करत नाहीत. हे LLM प्रतिसादाशी सुसंगत आहे, जो योग्यरित्या ओळखतो की संदर्भामध्ये अचूक तारीख माहितीचा अभाव आहे.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### हायब्रिड w/रिरँकिंग: LLM प्रतिसाद विश्लेषण
हायब्रिड w/रिरँकिंग उदाहरणामध्ये, LLM प्रतिसाद अतिरिक्त संदर्भ प्रदान करतो, ज्यामध्ये नमूद केले आहे की भाषण दिल्याच्या सहा दिवस आधी घटना घडली होती. याचा अर्थ असा की LLM भाषणाच्या वेळेनुसार आक्रमणाची तारीख अंदाजे सांगू शकतो, जरी अचूकतेसाठी भाषणाची नेमकी तारीख माहित असणे आवश्यक आहे.

हा प्रतिसाद संदर्भाच्या सूचनांचा वापर करून अधिक माहितीपूर्ण उत्तर देण्याची सुधारित क्षमता दर्शवतो. हे रिरँकिंगचे फायदे अधोरेखित करते, जिथे LLM अधिक संबंधित माहितीवर प्रवेश करून ती प्राधान्य देतो, ज्यामुळे अपेक्षित तपशील (उदा. आक्रमणाची तारीख) अधिक जवळपास मिळतो.

### स्रोत नोड्स विश्लेषण
या उदाहरणातील स्रोत नोड्समध्ये रशियाच्या आक्रमणाच्या वेळेचा संदर्भ समाविष्ट आहे, ज्यामध्ये विशेषतः नमूद केले आहे की ते भाषणाच्या सहा दिवस आधी घडले होते. जरी नेमकी तारीख स्पष्टपणे दिलेली नाही, तरी नोड्स तात्कालिक संदर्भ प्रदान करतात, ज्यामुळे LLM अधिक सूक्ष्म प्रतिसाद देऊ शकतो. या तपशीलाचा समावेश रिरँकिंग कसे LLM ला दिलेल्या संदर्भातून माहिती काढण्याची आणि अंदाजे सांगण्याची क्षमता सुधारते हे दर्शवतो, ज्यामुळे अधिक अचूक आणि माहितीपूर्ण प्रतिसाद मिळतो.


**टीप:**
या नोटबुकमध्ये, आम्ही NVIDIA API Catalog मधील NVIDIA NIM मायक्रोसर्व्हिसेस वापरल्या आहेत.  
वरील API, `NVIDIA (llms)`, `NVIDIAEmbedding`, आणि [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview).  
टीप, वरील API स्वतः-होस्ट केलेल्या मायक्रोसर्व्हिसेससाठी देखील समर्थन करू शकतात.

**उदाहरण:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण होणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
